In [1]:
import arviz as az
import pandas as pd
import pymc as pm

In [ ]:
data = pd.read_csv("./data/WaffleDivorce.csv", delimiter=";")

In [3]:
data.columns

Index(['Location', 'Loc', 'Population', 'MedianAgeMarriage', 'Marriage',
       'Marriage SE', 'Divorce', 'Divorce SE', 'WaffleHouses', 'South',
       'Slaves1860', 'Population1860', 'PropSlaves1860'],
      dtype='object')

In [7]:
with pm.Model() as m5_3:
    A = data["MedianAgeMarriage"]
    M = data["Marriage"]
    alpha = pm.Normal("alpha", mu=0, sigma=0.2)
    bA = pm.Normal("bA", mu=0, sigma=0.5)
    bM = pm.Normal("bM", mu=0, sigma=0.5)
    mu = alpha + bM * M + bA * A
    sigma = pm.Exponential("sigma", 1)
    pm.Normal("M", mu=mu, sigma=sigma)

    map_estimate = pm.find_MAP()
    trace = pm.sample(draws=2000, tune=1000)

Output()

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, bA, bM, sigma, M]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 10 seconds.
There were 6225 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [ ]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.041,0.192,-0.376,0.295,0.073,0.054,7.0,20.0,1.51
bA,-0.089,0.515,-0.822,0.586,0.219,0.164,7.0,29.0,1.58
bM,-0.063,0.292,-0.595,0.524,0.056,0.040,19.0,77.0,1.42
M[0],-3.571,15.336,-24.419,20.901,6.323,4.713,8.0,20.0,1.57
M[1],-3.923,16.272,-24.786,22.833,6.477,4.810,9.0,21.0,1.47
M[2],-3.412,15.251,-23.975,19.622,6.243,4.650,8.0,23.0,1.63
M[3],-3.745,16.154,-24.334,24.334,6.386,4.739,8.0,22.0,1.50
M[4],-3.303,15.882,-24.756,21.855,6.604,4.928,8.0,16.0,1.67
M[5],-3.665,16.074,-24.729,22.637,6.539,4.868,8.0,21.0,1.58
M[6],-3.470,16.072,-25.587,20.365,6.688,4.991,8.0,17.0,1.64
